In [1]:
# 修改预训练模型的缓存目录
import os

os.environ["MODELSCOPE_CACHE"] = r"G:\code\pretrain_model_dir\_modelscope"

In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('qwen/Qwen-14B-Chat-Int4', revision='v1.0.5')

2023-10-21 20:03:11,449 - modelscope - INFO - PyTorch version 2.1.0+cu118 Found.
2023-10-21 20:03:11,452 - modelscope - INFO - TensorFlow version 2.8.0 Found.
2023-10-21 20:03:11,453 - modelscope - INFO - Loading ast index from G:\code\pretrain_model_dir\_modelscope\ast_indexer
2023-10-21 20:03:11,561 - modelscope - INFO - Loading done! Current index file version is 1.9.3, with md5 dcadab61123e85c4b246f778e50d74cc and a total number of 943 components indexed
c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-21 20:03:14,565 - modelscope - INFO - Use user-specified model revision: v1.0.5


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

bin c:\tech\Anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Try importing flash-attention for faster inference...
CUDA extension not installed.
Loading checkpoint shards: 100%|██████████| 5/5 [00:04<00:00,  1.04it/s]


In [5]:
response, history = model.chat(tokenizer, "金子为什么浮在水面上", history=None)
print(response)

金子浮在水面上是因为它的密度比水小。密度是指物质的质量与其体积的比值，金子的密度约为19.32克/立方厘米，而水的密度约为1克/立方厘米。因此，当金子被放入水中时，它的质量比水大，但体积比水小，所以它会浮在水面上。


In [7]:
help(model.chat)

Help on method chat in module transformers_modules.Qwen-14B-Chat-Int4.modeling_qwen:

chat(tokenizer: transformers.tokenization_utils.PreTrainedTokenizer, query: str, history: Union[List[Tuple[str, str]], NoneType], system: str = 'You are a helpful assistant.', append_history: bool = True, stream: Union[bool, NoneType] = <object object at 0x000001F781D48DB0>, stop_words_ids: Union[List[List[int]], NoneType] = None, generation_config: Union[transformers.generation.configuration_utils.GenerationConfig, NoneType] = None, **kwargs) -> Tuple[str, List[Tuple[str, str]]] method of transformers_modules.Qwen-14B-Chat-Int4.modeling_qwen.QWenLMHeadModel instance



# int4 的好慢. 试试原始的

In [2]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
from modelscope import GenerationConfig

model_dir = snapshot_download('qwen/Qwen-14B-Chat', revision='v1.0.5')

2023-10-21 20:47:42,492 - modelscope - INFO - PyTorch version 2.1.0+cu118 Found.
2023-10-21 20:47:42,495 - modelscope - INFO - TensorFlow version 2.8.0 Found.
2023-10-21 20:47:42,495 - modelscope - INFO - Loading ast index from G:\code\pretrain_model_dir\_modelscope\ast_indexer
2023-10-21 20:47:42,620 - modelscope - INFO - Loading done! Current index file version is 1.9.3, with md5 dcadab61123e85c4b246f778e50d74cc and a total number of 943 components indexed
c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-21 20:47:45,408 - modelscope - INFO - Use user-specified model revision: v1.0.5


In [3]:
import torch
from modelscope import snapshot_download, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    False,
    True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)
model.generation_config = GenerationConfig.from_pretrained(model_dir)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


bin c:\tech\Anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████| 15/15 [00:13<00:00,  1.09it/s]


In [5]:
model.dtype, model.device

(torch.float16, device(type='cuda', index=0))

In [6]:
response, history = model.chat(tokenizer, "为什么金子浮在水面上", history=None)
print(response)

这是因为金子的密度小于水，所以它会浮在水面上。密度是物体质量和体积的比值，而水的密度大约为1克/立方厘米，金子的密度约为19.3克/立方厘米。由于金子的密度小于水，所以相同质量的金子和水相比，金子的体积更大，因此金子会漂浮在水面上。
需要注意的是，虽然金子浮在水面上，但是它的密度仍然大于许多其他物质，如铜、铁等，这些物质通常会在水中下沉。
